# Data Cleaning
- only keeping movies that were released (dropping anything pre-release)
- get rid of the columns that we wont use
- create binary response 

In [1]:
import pandas as pd
import numpy as np

In [30]:
# importing csv file
movies_metadata = pd.read_csv('movies_metadata.csv')

C:\Users\csky2\AppData\Local\Temp\ipykernel_4040\1536947796.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('movies_metadata.csv')


In [31]:
# only keeping movies that were released (dropping anything pre-release)
# drops about 400 movies
movies_metadata = movies_metadata.drop(np.where(movies_metadata['status'] != 'Released')[0])

In [32]:
# dropping 9 columns that we will not use
movies_metadata = movies_metadata.drop(['adult', 'homepage', 'id', 'imdb_id', 'original_title', 'poster_path', 'status', 'video', 'overview', 'production_companies', 'production_countries', 'spoken_languages', 'tagline'], axis=1)

In [33]:
# cleaning belongs_to_collection - making it binary
movies_metadata.loc[~movies_metadata.belongs_to_collection.isnull(), 'belongs_to_collection'] = 1  # not nan
movies_metadata.loc[movies_metadata.belongs_to_collection.isnull(), 'belongs_to_collection'] = 0

In [34]:
movies_metadata.columns

Index(['belongs_to_collection', 'budget', 'genres', 'original_language',
       'popularity', 'release_date', 'revenue', 'runtime', 'title',
       'vote_average', 'vote_count'],
      dtype='object')

In [35]:
print('num of movies that made money:', movies_metadata[movies_metadata['revenue']!=0].shape[0])
print('num of movies that didnt make money:', movies_metadata[movies_metadata['revenue']==0].shape[0])

num of movies that made money: 7395
num of movies that didnt make money: 37619


In [36]:
# making y binary
movies_classification = movies_metadata.copy()
movies_classification.loc[movies_metadata['revenue']!=0, 'revenue'] = 1
movies_classification.loc[movies_metadata['revenue']==0, 'revenue'] = 0

In [37]:
def get_dummies_complex(column_name):
    column_name = column_name.apply(lambda x:eval(x))
    dict_items = []
    for list_of_dicts in column_name:
        for dictionary in list_of_dicts:
            if dictionary['name'] not in dict_items:
                dict_items.append(dictionary['name'])
    for item in dict_items:
        movies_classification[item] = 0
    for ind in movies_classification.index:
        items = column_name[ind]
        for dictionary in items:
            movies_classification.loc[ind, dictionary['name']] = 1
    return movies_classification

In [38]:
# making dummies for genres
get_dummies_complex(movies_classification.genres)

,belongs_to_collection,budget,genres,original_language,popularity,release_date,revenue,runtime,title,vote_average,...,Horror,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie
0,1,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,21.946943,1995-10-30,1.0,81.0,Toy Story,7.7,...,0,0,0,0,0,0,0,0,0,0
1,0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,17.015539,1995-12-15,1.0,104.0,Jumanji,6.9,...,0,0,0,0,0,0,0,0,0,0
2,1,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,11.7129,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,...,0,0,0,0,0,0,0,0,0,0
3,0,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,3.859495,1995-12-22,1.0,127.0,Waiting to Exhale,6.1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,"[{'id': 35, 'name': 'Comedy'}]",en,8.387519,1995-02-10,1.0,106.0,Father of the Bride Part II,5.7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",fa,0.072051,NaN,0.0,90.0,Subdue,4.0,...,0,0,0,0,0,0,0,0,0,0
45462,0,0,"[{'id': 18, 'name': 'Drama'}]",tl,0.178241,2011-11-17,0.0,360.0,Century of Birthing,9.0,...,0,0,0,0,0,0,0,0,0,0
45463,0,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,0.903007,2003-08-01,0.0,90.0,Betrayal,3.8,...,0,0,0,0,0,0,0,0,0,0
45464,0,0,[],en,0.003503,1917-10-21,0.0,87.0,Satan Triumphant,0.0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
movies_classification.drop('genres', axis=1, inplace = True)

In [41]:
# original language -> english vs not
ind = movies_classification.loc[movies_classification.original_language.isin(['en']), 'original_language'].index
movies_classification['english'] = 0
for movie in ind:
    movies_classification.loc[ind, 'english'] = 1

In [42]:
movies_classification.drop('original_language', axis = 1, inplace = True)

In [45]:
movies_classification.release_date = pd.to_datetime(movies_classification.release_date)
movies_classification.release_date = pd.to_numeric(movies_classification.release_date)

In [46]:
movies_classification.shape

(45014, 30)

In [47]:
movies_classification.to_csv('movies_classification.csv')